In [ ]:
import pandas as pd
from hs_gimme.constants.objects.req import Req
from collections import defaultdict

In [ ]:
REQ_FIELDS = [Req.COUNTRY, Req.INDUSTRY, Req.TOP_CATEGORY,
              Req.SENIORITY_LEVEL, Req.JOB_EDUCATION]

In [1]:
#df = pd.read_csv('/Users/dima/Downloads/fetch_evaluation_results_all_clients.csv')[lambda x: x['filter'] == 'ALL'].sample(10000)
df = pd.read_csv('/Users/dima/hiredscore/brain_models_creation/tree_blenders/hs_tree_blenders_creation/fast_fetch_evaluation/results/fetch_evaluation_results.csv')
df.shape

(2000, 8)

In [2]:
df['filter'].value_counts()

ProfessionsQueryFilter            1000
DetailedProfessionsQueryFilter    1000
Name: filter, dtype: int64

In [3]:
df = df[df['filter'] == 'ProfessionsQueryFilter']

In [ ]:
def add_req_fields(df, env):
    reqs = {}
    num_of_accounts = len(df['account_id'].unique())
    for i, (account, account_df) in enumerate(df.groupby('account_id')):
        try:
            mongo = get_mongo_client_db(env, account)

            req_ids = list(account_df['req_id'].unique())
            print(f'{i}/{num_of_accounts} {account} {len(req_ids)} reqs')

            for r in mongo.req.find({'_id': {'$in': req_ids}}, REQ_FIELDS):
                reqs[r['_id']] = r
        except Exception as e:
            print(account, e)
    for f in REQ_FIELDS:
        df[f] = df['req_id'].apply(lambda r: reqs.get(r, {}).get(f)).fillna('NONE')

In [ ]:
def add_apps_fields(df, env):
    apps = {}
    num_of_accounts = len(df['account_id'].unique())
    for i, (account, account_df) in enumerate(df.groupby('account_id')):
        try:
            mongo = get_mongo_client_db(env, account)

            talent_ids = list(account_df['current_talent_id'].unique())
            print(f'{i}/{num_of_accounts} {account} {len(talent_ids)} talents')

            for r in mongo.application.find({'current_talent_id': {'$in': talent_ids}}, ['samurai_json.experience', 'req_id', 'current_talent_id']):
                apps[r['req_id'] + r['current_talent_id']] = r
        except Exception as e:
            print(account, e)
            
    
        df['current_seniority'] = df.apply(lambda r: (apps.get(r['req_id'] + r['current_talent_id'], {}).get('samurai_json', {}).get('experience') or [{}])[0].get('seniority'), axis=1).fillna('NONE')
        df['current_title'] = df.apply(lambda r: (apps.get(r['req_id'] + r['current_talent_id'], {}).get('samurai_json', {}).get('experience') or [{}])[0].get('title'), axis=1).fillna('NONE')

In [ ]:
add_req_fields(df, 'production')

In [ ]:
add_apps_fields(df, 'production')

In [ ]:
df['current_seniority'].value_counts()

In [ ]:
m = lambda x: x['seniority_level'].isin({'entry_level', 'junior', 'senior'}) & (x['group_name'] == 'offer')

In [ ]:
df[m].groupby(['seniority_level', 'current_seniority']).size()

In [4]:
df['group_name'].value_counts()

Random from other reqs    353
rejected                  327
offer                     320
Name: group_name, dtype: int64

In [7]:
mask = lambda x: (x['seniority_level'] == 'entry_level') & \
                 (x['group_name'] == 'Random from other reqs') & \
                 (x['filter'] == 'ALL') & \
                 (x['predicted'] == 0)

In [8]:
mask = lambda x: (x['group_name'] == 'Random from other reqs') & \
                 (x['predicted'] == 1)

In [ ]:
df['title_is_none'] = df['current_title'] == 'NONE'

In [ ]:
sdf = df[lambda x: (x['seniority_level'] == 'senior') & \
   (x['filter'] == 'ALL') & 
                 (x['group_name'] == 'offer')]

In [ ]:
sdf.groupby('title_is_none')['predicted'].size()

In [9]:
df[mask].sample(30)[['account_id', 'req_id', 'current_talent_id']].values

array([['alabama', 'ABV1906625', '5c543b7c-60a4-4b20-9446-73661e428e1b'],
       ['atlanta', 'AAL33485', '928374ca-b915-4198-abee-95fb874cc34e'],
       ['sofia', 'SBY106084', '20072a8e-5142-4aa1-aaca-818ae659cdb3'],
       ['sofia', 'SBY108641', '799a40ba-71aa-472a-9bc8-f1d5e7a372db'],
       ['detroit_avature', 'DLA43181',
        'a9e5fd72-31a4-4054-af0c-f6c38a6a1b11'],
       ['tampa', 'TYOR0068716', '1624278e-c028-44ed-909e-e264c059c755'],
       ['arizona_wd', 'ACNR00110752',
        '49491dce-8306-432c-a667-a571c43a30d2'],
       ['atlanta', 'AAL49384', 'bcebfa30-4b5f-4a49-b087-36142fa0a008'],
       ['sofia', 'SBY158401', 'fc692a48-9f88-45d4-85be-2853e3f05fbd'],
       ['paris', 'PFZ4830785', '55ff17ae-6d6b-41e7-a8ca-690c776a673a'],
       ['sofia', 'SBY162222', '9d039011-b46c-46cd-936d-40046e72b722'],
       ['sofia', 'SBY83802', '86d6a9ed-bf53-4dc3-b33b-041ed297e070'],
       ['bogota', 'BAC00000117303',
        'f12c49db-3708-41f9-9d63-18b8fc6e5f82'],
       ['moscow', 'MCY8

In [ ]:
path = '/Users/dima/hiredscore/brain_models_creation/tree_blenders/hs_tree_blenders_creation/fast_fetch_evaluation/data/fetch_evaluation_dataset.json'

In [ ]:
df[df['filter'] == 'ALL'].shape

In [ ]:
m = lambda x: (x['seniority_level'].isin({'entry_level', 'senior'})) & \
                 (x['filter'] == 'ALL')
df[m].shape, df[m]['seniority_level'].value_counts()

In [ ]:

dataset = defaultdict(lambda: defaultdict(list))
for account_id, req_id, current_talent_id, group_name in df[m].sample(1000)[['account_id', 'req_id' ,'current_talent_id', 'group_name']].values:
    dataset[account_id][req_id].append({
                            'current_talent_id': current_talent_id,
                            'evaluation_group': group_name,
                            'sources': ['application']
                        })

In [ ]:
import json

In [ ]:
json.dump(dataset, open(path, 'w'))

In [ ]:
df[(df['current_seniority'] == 'NONE') & (df['group_name'] == 'offer')].tail(10)

In [ ]:
mongo = gmcdb('production', 'sofia')

In [ ]:
app = mongo.application.find_one({'req_id': 'SBY162473', 'current_talent_id': 'e73f44ff-9118-407c-b9bf-b221e91181aa'})

In [ ]:
sorted(app['samurai_json']['experience'][0].keys())

In [ ]:
app

In [ ]:
app = mongo.application.find_one({'current_talent_id': '9254854d-05eb-4cdf-935f-634ed16f1aaf'})
len(app)

In [ ]:
app['date_applied']

In [ ]:
df[m][lambda x: (x['current_seniority'] == 'entry_level') & (x['seniority_level'] == 'senior')].head()

In [ ]:
mongo = gmcdb('production', 'cairo')

In [ ]:
app = mongo.application.find_one({'req_id': 'CSG209037', 'current_talent_id': '3c5b7a37-c295-4d2e-950d-241387e44315'})

In [ ]:
len(app)

In [ ]:
app['samurai_json']['experience'][0]

In [10]:
mongo = gmcdb('production', 'moscow')

2023-09-20T08:51:34.393299Z [info     ] Mongos instances selected      environment=local file_path=/Users/dima/hiredscore/gimme/hs_gimme/db_facade/connection_string_builder.py function_name=get_selected_mongos_instances hostname=Dima-Shulga-MacBook-Pro hosts=['applicativedb-prod-mongos-0.omcomcom.com', 'applicativedb-prod-mongos-2.omcomcom.com'] line_number=38 module=hs_gimme.db_facade.connection_string_builder pid=94687


In [11]:
req = mongo.req.find_one('MCY863638')

In [13]:
sorted(req.keys())

['_created_at',
 '_id',
 '_updated_at',
 '_version',
 'account_id',
 'activation_data',
 'address',
 'ats_salary_data',
 'basic_qualifications_skills',
 'calculated_required_skills',
 'categories',
 'category_score',
 'certifications',
 'city',
 'clean_job_title',
 'close_date',
 'condition_based_experience_requirements',
 'country',
 'country_name',
 'date_posted',
 'deep_experience',
 'deep_experience_profession',
 'displayed_req_addresses',
 'education',
 'english_job_title',
 'experience',
 'external_id',
 'external_recruiters',
 'external_status',
 'flsa_category',
 'important_words',
 'industry',
 'integration_name',
 'is_enabled',
 'is_gradable',
 'is_intern',
 'is_recent_grad',
 'is_visa_required',
 'job_code',
 'job_create_date',
 'job_education',
 'job_listing',
 'job_listing_sentences',
 'job_title',
 'job_type',
 'knowledge_base_skills',
 'languages',
 'last_edit_date',
 'main_requirements_tracking',
 'management_requirements',
 'max_distance',
 'max_salary',
 'max_years_of

In [14]:
req['top_category']

'Medical'

In [15]:
req['sub_category']

'Nursing'

In [16]:
from hs_fetch.fast_fetch.fast_fetch_req_transformers import _get_main_top_professions_scores_data

2023-09-20T08:53:30.394903Z [info     ] Explainable job type: loaded aliases to job types environment=local file_path=/Users/dima/hiredscore/grading/algorithms/hs_algorithms/explainable_job_type/explainable_job_type.py function_name=load_job_type_data hostname=Dima-Shulga-MacBook-Pro line_number=109 module=hs_algorithms.explainable_job_type.explainable_job_type num_aliases__numeric=365 num_job_types__numeric=214 pid=94687


In [18]:
x = _get_main_top_professions_scores_data(req)

In [19]:
len(x)

4

In [20]:
x[0]

{'Administrative': 0.6500558853149414, 'Medical': 0.8910780549049377}

In [21]:
x[1]

0.8910780549049377

In [22]:
x[2]

{228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 242,
 243,
 244,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 266,
 537}

In [24]:
req['professions_scores'][482]

0.8910780549049377

In [26]:
from hs_gimme.constants.professions import PROFESSIONS_MAPPING

In [27]:
import numpy as np

In [28]:
for p in np.argsort(req['professions_scores']):
    print(PROFESSIONS_MAPPING[p])

Sales#Travel_Agent_=_Ticket_Sales#Hotel Sales
Sales#Industry#Finance
Engineering#Bio-Engineering#Medical Devices Engineering
Maintenance_&_Repair#Equipment_Install_=_Maintenance_=_Repair#Rotating Equipment
Engineering#Academics
Extras#Weather_Forecasting
Legal#Attorney#Injuries
Extras#Fundraising
Extras#Agriculture
Education#Classroom_Teaching#Assistant
Education#School_=_College_Administration#Alumni Relations
Sales#Federal
Extras#Religion#Pastor
Design_&_Creative#Photography_&_Videography#Photography
Education#Special_Education#School Psychologyst
Engineering#Electrical_=_Electronic_Engineering#PCB Design
Education#Classroom_Teaching#Math Teacher
Extras#Fundraising#Grant Writing
Sales#Sales_System_Administration
Sales#Online_Sales
Extras#NGO#Community Organizer
IT_&_Software_Development#Telecommunications_Administration_=_Management
Sales#Outside_Sales
Legal#Attorney#Bankruptcy_Attorney
IT_&_Software_Development#Data_Scientist_=_Applicative_Research#IT Research 
Sales#Industry#Metal


In [31]:
app = mongo.application.find_one({'current_talent_id': '877e845f-a43d-4d65-8034-fb9cafd5cd51'})

In [38]:
PROFESSIONS_MAPPING[432]

'Manufacturing#Operations_=_Plant_Management'

In [39]:
for x, y in [[445, 0.3759063184261322],
    [433, 0.3207433521747589],
    [432, 0.24318262934684753],
    [421, 0.210514634847641],
    [408, 0.1503463238477707],
    [545, 0.13750621676445007],
    [409, 0.13055257499217987],
    [506, 0.12325996905565262],
    [420, 0.11994505673646927],
    [66, 0.11927130073308945]]:
    
    print(PROFESSIONS_MAPPING[x])

Manufacturing#Production_Supervisor
Manufacturing#Other
Manufacturing#Operations_=_Plant_Management
Maintenance_&_Repair#Facilities_Management_=_Maintenance
Logistics_&_Transportation#Shipping_and_Receiving_=_Warehousing#Logistics Manager
Supply_Chain_Management#Purchasing
Logistics_&_Transportation#Shipping_and_Receiving_=_Warehousing
Sales#Industry#Automotive_Sales
Maintenance_&_Repair#Equipment_Install_=_Maintenance_=_Repair
Building_&_Construction#Surveying#Project Management


In [33]:
app['samurai_json']

{'experience': [{'title': 'Director of Operations',
   'company_name': 'Ozarks Medical Center',
   'start_date': datetime.datetime(2018, 2, 1, 0, 0),
   'current_job': True,
   'end_date': None,
   'project_role': None,
   'project_name': None,
   'project_client': None,
   'project_skills': None,
   'description': '1. Director of Operations- Feb 2018-Present',
   'description_short': '',
   'is_project': False,
   'is_leadership': True,
   'is_volunteering': False,
   'is_internship': False,
   'clean_english_title': 'director operations',
   'duration_in_month': 55,
   'company_normalized_name': 'Ozarks Medical Center',
   'company_details': None,
   'is_related_to_client_company': False,
   'is_contractor': False,
   'same_company': True,
   'is_management_position': False,
   'professions': [[226, 0.5851806350167741],
    [81, 0.5534406342518942],
    [433, 0.4878015455726963],
    [42, 0.48006935658726396],
    [78, 0.4678443037027391],
    [77, 0.46784299025493836],
    [43, 0.45

In [40]:
reqs = list(mongo.req.find({'top_category': 'Medical'}).limit(10))

In [41]:
len(reqs)

10

In [42]:
from hs_fetch.fast_fetch.refactored.fetch_data_adapter import get_data_adapter

/Users/dima/.virtualenvs/hs/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
/Users/dima/.virtualenvs/hs/lib/python3.9/site-packages/xgboost/compat.py:31: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


In [ ]:
    data_adapter = get_data_adapter(
        'moscow', 'production', None, None,
        es_proxy=persistor, mongo_proxy=fast_fetch_queries, use_external_data=use_external_data
    )

In [ ]:
indexed_req_data, external_req_data = self.data_adapter.get_req_data(req_id)